# 1. Transfer learning (story - aila)

In [221]:
# Creating story pairs

import pandas 
from sklearn.model_selection import train_test_split

data = pandas.read_csv("C:/Users/Thiloshon/PycharmProjects/analogical-transfer-learning/02_aesop_fables_annotated.csv")
stories = list(data['story'])
tags = list(data['tag'])

pairs = []
unpairs = []

for l1 in range(0, len(stories)):
    for l2 in range(l1, len(stories)):
        if l1 != l2 and tags[l1]==tags[l2]:
            pairs.append((stories[l1], stories[l2], 1))
        elif l1 != l2 and tags[l1]!=tags[l2]:
            unpairs.append((stories[l1], stories[l2], 0))
            
            
import random

random.shuffle(unpairs)
random.shuffle(pairs)

tot_pairs = pairs[0:544] + unpairs[0:544]
train, dev = train_test_split(tot_pairs, test_size=0.20)

In [222]:
len(pairs)

544

In [223]:
len(train)


870

In [224]:
# Learning
from sentence_transformers import InputExample
from sentence_transformers import models, losses, datasets
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.cross_encoder.evaluation import CEBinaryAccuracyEvaluator
import math
import matplotlib.pyplot as plt
from sentence_transformers import CrossEncoder
from sklearn.metrics import accuracy_score

train_samples = [InputExample(texts=[a[0], a[1]], label=int(a[2])) for a in train] 

model = CrossEncoder('distilroberta-base', num_labels=1)

train_dataloader = datasets.NoDuplicatesDataLoader(train_samples, batch_size=32)

num_epochs = 5

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)

history = model.fit(train_dataloader  = train_dataloader,
                    epochs=num_epochs,
                    evaluation_steps=int(len(train_dataloader)*0.1),
                    warmup_steps=warmup_steps,
                    use_amp=False,
                   )

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/27 [00:00<?, ?it/s]

Iteration:   0%|          | 0/27 [00:00<?, ?it/s]

Iteration:   0%|          | 0/27 [00:00<?, ?it/s]

Iteration:   0%|          | 0/27 [00:00<?, ?it/s]

Iteration:   0%|          | 0/27 [00:00<?, ?it/s]

In [ ]:
#additional training

num_epochs = 1

history = model.fit(train_dataloader  = train_dataloader,
                    epochs=num_epochs,
                    evaluation_steps=int(len(train_dataloader)*0.1),
                    warmup_steps=warmup_steps,
                    use_amp=False,
                   )

In [225]:
dev_x = [(a[0], a[1]) for a in dev]
dev_y = [a[2] for a in dev]
a = model.predict(dev_x)

a

array([0.3804929 , 0.44115415, 0.5037675 , 0.5032178 , 0.46191627,
       0.5032922 , 0.5777111 , 0.5014547 , 0.6060726 , 0.4776313 ,
       0.44060907, 0.51764613, 0.45569417, 0.48611835, 0.49182463,
       0.49153817, 0.5119049 , 0.47792956, 0.511141  , 0.44472578,
       0.54157466, 0.4522344 , 0.52562857, 0.42123574, 0.38834342,
       0.47088927, 0.44302174, 0.46462482, 0.4734608 , 0.51279604,
       0.4453686 , 0.45378688, 0.44707826, 0.42289713, 0.44101983,
       0.4464787 , 0.37810248, 0.53672385, 0.51047903, 0.44783714,
       0.55179536, 0.4779942 , 0.4593432 , 0.48879158, 0.43366158,
       0.48341691, 0.47425634, 0.38674322, 0.53586817, 0.5514141 ,
       0.35558027, 0.49307546, 0.5607087 , 0.49468017, 0.42203146,
       0.4581029 , 0.5025715 , 0.51077646, 0.4362424 , 0.3333053 ,
       0.39356205, 0.52067477, 0.5091464 , 0.4563414 , 0.50054955,
       0.42373574, 0.4401554 , 0.41290957, 0.4447588 , 0.47554126,
       0.3686713 , 0.48349014, 0.44331217, 0.4914554 , 0.47157

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

threshold = 0.49

preds = [int(ass > threshold) for ass in a]
print(accuracy_score(dev_y, preds))
print(f1_score(dev_y, preds, average="macro"))

In [226]:
### Transfer EVALUATION on analogy type #####


test_data = pandas.read_csv("C:/Users/Thiloshon/PycharmProjects/analogical-transfer-learning/aesop_morals_pairs.csv")[0:44]

test_x = list(zip(list(test_data["STORY 1"]), list(test_data["STORY 2"])))

test_pred = model.predict(test_x)

In [31]:
# oo = "SAA Score	DAA Score	RA Score	SA Score	MP Score	LS Score"

test_y = list(test_data["LS Score"])

preds2 = [int(ass > threshold) for ass in test_pred]
print(accuracy_score(test_y, preds2))
print(f1_score(test_y, preds2, average="macro"))

0.25
0.2


In [ ]:
### Transfer EVALUATION on AILA data #####

# pair creation

file1 = open('C:/Users/Thiloshon/PycharmProjects/analogical-transfer-learning/AILA_2019_dataset/relevance_judgments_priorcases.txt', 'r')
lines = file1.readlines()

m = [a.replace("\n", "").split(" ") for a in lines]

cases = {}

for li in m:
    if li[3] == "1":
        gg = cases.get(li[0], [])
        gg.append(li[2])
        cases[li[0]] = gg

un_cases = {}

for li in m:
    if li[3] == "0":
        gg = un_cases.get(li[0], [])
        gg.append(li[2])
        un_cases[li[0]] = gg

        
pairs_1 = []

for a in cases:
    cs = cases[a]
    
    for l1 in range(0, len(cs)):
        for l2 in range(l1, len(cs)):
            if l1 != l2:
                data1 = ""
                data2 = ""
                with open('AILA_2019_dataset/Object_casedocs/' + cs[l1] + ".txt", 'r') as file:
                    data1 = file.read().replace('\n', '')[500: 2500] + cs[l1]
                with open('AILA_2019_dataset/Object_casedocs/' + cs[l2] + ".txt", 'r') as file:
                    data2 = file.read().replace('\n', '')[500: 2500] + cs[l2]
                pairs_1.append((data1, data2, 1))
                
un_pairs_1 = []

for a in un_cases:
    cs = un_cases[a][0:6]
    
    for l1 in range(0, len(cs)):
        for l2 in range(l1, len(cs)):
            
            if l1 != l2:
                datas1 = ""
                datas2 = ""
                with open('AILA_2019_dataset/Object_casedocs/' + cs[l1] + ".txt", 'r') as file:
                    datas1 = file.read().replace('\n', '')[500: 2500] + cs[l1]
                with open('AILA_2019_dataset/Object_casedocs/' + cs[l2] + ".txt", 'r') as file:
                    datas2 = file.read().replace('\n', '')[500: 2500] + cs[l2]
                    
                un_pairs_1.append((datas1, datas2, 0))
    
un_pairs_1 = un_pairs_1[0:640]
tot_pair_1 = pairs_1 + un_pairs_1
len(tot_pair_1)

In [ ]:
# Test predition

test_x = [(a[0], a[1]) for a in tot_pair_1]
test_y = [a[2] for a in tot_pair_1]
         
a2 = model.predict(test_x)


In [ ]:
preds = [int(ass > 0.45) for ass in a2]
print(accuracy_score(test_y, preds))
print(f1_score(test_y, preds, average="macro"))

# Transfer learning AILA - story

In [2]:
# Case pairs creation

from sklearn.model_selection import train_test_split
import pandas
import random

file1 = open('C:/Users/Thiloshon/PycharmProjects/analogical-transfer-learning/AILA_2019_dataset/relevance_judgments_priorcases.txt', 'r')
lines = file1.readlines()

m = [a.replace("\n", "").split(" ") for a in lines]

cases = {}

for li in m:
    if li[3] == "1":
        gg = cases.get(li[0], [])
        gg.append(li[2])
        cases[li[0]] = gg

un_cases = {}

for li in m:
    if li[3] == "0":
        gg = un_cases.get(li[0], [])
        gg.append(li[2])
        un_cases[li[0]] = gg

        
pairs_1 = []

for a in cases:
    cs = cases[a]
    
    for l1 in range(0, len(cs)):
        for l2 in range(l1, len(cs)):
            if l1 != l2:
                data1 = ""
                data2 = ""
                with open('AILA_2019_dataset/Object_casedocs/' + cs[l1] + ".txt", 'r') as file:
                    data1 = file.read().replace('\n', '')[500: 2500] + cs[l1]
                with open('AILA_2019_dataset/Object_casedocs/' + cs[l2] + ".txt", 'r') as file:
                    data2 = file.read().replace('\n', '')[500: 2500] + cs[l2]
                pairs_1.append((data1, data2, 1))
                
un_pairs_1 = []

for a in un_cases:
    cs = un_cases[a][0:6]
    
    for l1 in range(0, len(cs)):
        for l2 in range(l1, len(cs)):
            
            if l1 != l2:
                datas1 = ""
                datas2 = ""
                with open('AILA_2019_dataset/Object_casedocs/' + cs[l1] + ".txt", 'r') as file:
                    datas1 = file.read().replace('\n', '')[500: 2500] + cs[l1]
                with open('AILA_2019_dataset/Object_casedocs/' + cs[l2] + ".txt", 'r') as file:
                    datas2 = file.read().replace('\n', '')[500: 2500] + cs[l2]
                    
                un_pairs_1.append((datas1, datas2, 0))
print(len(un_pairs_1))
    
un_pairs_1 = un_pairs_1[0:640]

tot_pair_1 = pairs_1 + un_pairs_1

train, dev = train_test_split(tot_pair_1, test_size=0.20)
random.shuffle(tot_pair_1)

len(tot_pair_1)

750


1280

In [4]:
len(train)

1024

In [122]:
# Leanring similar cases

from sentence_transformers import InputExample
from sentence_transformers import models, losses, datasets
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.cross_encoder.evaluation import CEBinaryAccuracyEvaluator
import math
import matplotlib.pyplot as plt
from sentence_transformers import CrossEncoder
from sklearn.metrics import accuracy_score

train_samples = [InputExample(texts=[a[0], a[1]], label=int(a[2])) for a in train] 

model = CrossEncoder('sentence-transformers/paraphrase-MiniLM-L6-v2', num_labels=1)

train_dataloader = datasets.NoDuplicatesDataLoader(train_samples, batch_size=32)

num_epochs = 1

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)

history = model.fit(train_dataloader  = train_dataloader,
                    epochs=num_epochs,
                    evaluation_steps=int(len(train_dataloader)*0.1),
                    warmup_steps=warmup_steps,
                    use_amp=False,
                   )

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/paraphrase-MiniLM-L6-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/32 [00:00<?, ?it/s]

In [179]:
history = model.fit(train_dataloader  = train_dataloader,
                    epochs=num_epochs,
                    evaluation_steps=int(len(train_dataloader)*0.1),
                    warmup_steps=warmup_steps,
                    use_amp=False,
                   )

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/32 [00:00<?, ?it/s]

In [180]:
# dev data evalatuion

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

dev_x = [(a[0], a[1]) for a in dev]
dev_y = [a[2] for a in dev]
a = model.predict(dev_x)

In [214]:
threshold = 0.453

preds = [int(ass > threshold) for ass in a]
print(accuracy_score(dev_y, preds))
print(f1_score(dev_y, preds, average="macro"))

0.73828125
0.7332960146786709


In [181]:
# Transfer learning evaluation

test_data = pandas.read_csv("C:/Users/Thiloshon/PycharmProjects/analogical-transfer-learning/aesop_morals_pairs.csv")[0:44]

test_x = list(zip(list(test_data["STORY 1"]), list(test_data["STORY 2"])))

test_pred = model.predict(test_x)

In [220]:
# oo = "SAA Score	DAA Score	RA Score	SA Score	MP Score	LS Score"

test_y = list(test_data["LS Score"])

preds2 = [int(ass > threshold) for ass in test_pred]
print(accuracy_score(test_y, preds2))
print(f1_score(test_y, preds2, average="macro"))

0.25
0.2


# Annotation IAA

In [ ]:
import sklearn
from sklearn.metrics import cohen_kappa_score


data = pandas.read_csv("aesop_morals_com_JUMBLED_v4_stats.csv")


In [ ]:
cohen_kappa_score(list(data["LS_SHON"]), list(data["MP_F"]))

In [ ]:
import statsmodels
from statsmodels.stats.inter_rater import fleiss_kappa

from statsmodels.stats import inter_rater as irr


agg = irr.aggregate_raters(data[["LS_JAY", "LS_SHON", "LS_F"]]) # returns a tuple (data, categories)
irr.fleiss_kappa(agg[0], method='fleiss')